## 2. Report Generation

The final phase of SDMT involves aggregating evidence, validating the metrics reflected by the evidence we collected, and displaying this information in a report.

#### Preliminaries

In [ ]:
# Preliminaries for loading the package locally
import os
import sys

def package_root() -> str:
    """Resolve the path to the project root."""
    return os.path.abspath(os.path.join(os.getcwd(), "..", "src/"))

sys.path.append(package_root())
sys.path.append(os.getcwd())

In [ ]:
import os
from pathlib import Path

# The path at which media is stored
REPORTS_DIR = Path(os.getcwd()) / "reports"
os.makedirs(REPORTS_DIR, exist_ok=True)

#### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces.

In [ ]:
import mlte

store_path = os.path.join(os.getcwd(), "store")

mlte.set_model("IrisClassifier", "0.0.1")
mlte.set_artifact_store_uri(f"local://{store_path}")

#### Set Up Conditions

In MLTE, a `Condition` indicates a validation to be performed on a Property.

In [ ]:
from mlte.spec import Spec, Condition
from mlte.measurement import ExternalMeasurement
from mlte.measurement.result import Real, Image
from confusion_matrix import ConfusionMatrix
from mlte.measurement.cpu import LocalProcessCPUUtilization
from mlte.measurement.storage import LocalObjectSize

# Load the specification
spec = Spec.load()


In [ ]:
# Add the conditions to validate.
spec.add_condition("TaskEfficacy", Condition(ExternalMeasurement("accuracy", Real), "greater_or_equal_to", 0.9))
spec.add_condition("TaskEfficacy", Condition(ExternalMeasurement("confusion matrix", ConfusionMatrix).metadata, "misclassification_count_less_than", 2))
spec.add_condition("TaskEfficacy", Condition(ExternalMeasurement("class distribution", Image).metadata, "ignore", "Inspect the image."))
spec.add_condition("StorageCost", Condition(LocalObjectSize("model size").metadata, "less_than", 3000))
#spec.add_condition("TrainingMemoryCost", Condition(ExternalMeasurement("accuracy", Real), "greater_or_equal_to", 0.9))
spec.add_condition("TrainingComputeCost", Condition(LocalProcessCPUUtilization("training cpu").metadata, "max_utilization_less_than", 5.0))

spec.save()

### Validate Results and Bind `Result`s to `Property`s

In [ ]:
from mlte.measurement.result import Integer, Real, Image
from mlte.measurement.cpu import CPUStatistics
from confusion_matrix import ConfusionMatrix
spec.add_result(Integer.load("model size"))
spec.add_result(CPUStatistics.load("training cpu"))
spec.add_result(Real.load("accuracy"))
spec.add_result(ConfusionMatrix.load("confusion matrix"))
spec.add_result(Image.load("class distribution"))

In [ ]:
# Validate conditions and get bound details.
bound_spec = spec.validate_and_bind()

# BoundSpec also supports persistence
bound_spec.save()

#### Generate a Report

The final step of SDMT involves the generation of a report to communicate the results of model evaluation.

In [ ]:
import time
from mlte.report import Report, Dataset, User, UseCase, Limitation

def unix_timestamp() -> str:
    return f"{int(time.time())}"

def build_report() -> Report:
    report = Report()
    report.metadata.project_name = "IrisClassificationProject"
    report.metadata.authors = ["Kyle Dotterrer", "Kate Maffey"]
    report.metadata.source_url = "https://github.com/mlte-team"
    report.metadata.artifact_url = "https://github.com/mlte-team"
    report.metadata.timestamp = unix_timestamp()

    report.model_details.name = "IrisClassifier"
    report.model_details.overview = "A model that distinguishes among three (3) types of irises."
    report.model_details.documentation = "This is a simple model that can distinguish between the setosa, versicolour, and virginica species of Iris based on physical characteristics."

    report.model_specification.domain = "Classification"
    report.model_specification.architecture = "Decision Tree"
    report.model_specification.input = "Vector[4]"
    report.model_specification.output = "Binary"
    report.model_specification.data = [
        Dataset("Dataset0", "https://github.com/mlte-team", "This is one training dataset."),
        Dataset("Dataset1", "https://github.com/mlte-team", "This is the other one we used."),
    ]

    report.considerations.users = [
        User("Botanist", "A professional botanist."),
        User("Explorer", "A weekend-warrior outdoor explorer."),
    ]
    report.considerations.use_cases = [
        UseCase("Personal Edification", "Quench your curiosity: what species of iris IS that? Wonder no longer.")
    ]
    report.considerations.limitations = [
        Limitation(
            "Low Training Data Volume",
            """
            This model was trained on a low volume of training data.
            """,
        ),
    ]
    return report

In [ ]:
from mlte.report import render

# Build the base report
report = build_report()
# Attach the bound specification
report.spec = bound_spec

# Save the report as an HTML document
report.to_html(REPORTS_DIR / "report.html", local=True)